In [3]:
!pip install transformers peft datasets bitsandbytes huggingface_hub wandb -qqq

In [4]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)

from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from datasets import load_dataset
import bitsandbytes
import torch
import wandb

In [ ]:
# 허깅페이스 로그인
from huggingface_hub import login

# 허깅페이스 허브 로그인
token = "****"  # 허깅페이스 액세스 토큰 입력
login(token=token)

In [6]:
# 모델과 데이터셋 이름 설정
base_model = 'mistralai/Mistral-7B-v0.1'
dataset_name = 'daekeun-ml/naver-news-summarization-ko'
new_model = 'mistral-7b-konews-summarizer'

In [7]:
# 데이터셋 로드 및 샘플링
dataset = load_dataset(dataset_name, split='train')
print(f"Original dataset size: {len(dataset)}")

# 2000개 샘플로 제한 (랜덤 샘플링)
dataset = dataset.shuffle(seed=42).select(range(4000))
print(f"Sampled dataset size: {len(dataset)}")
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Original dataset size: 22194
Sampled dataset size: 4000
Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
    num_rows: 4000
})


In [8]:
## 모델 및 토크나이저 준비
# 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # eos 토큰을 패딩 토큰으로 설정
tokenizer.padding_side = 'right'           # 오른쪽 패딩

# 환경 변수 설정으로 tokenizer 경고 제거
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [9]:
# instruction 템플릿 정의
def create_prompt_template(document):
    return f"""아래 뉴스 기사를 요약해주세요:

기사 내용:
{document}

요약:"""

def create_completion_template(summary):
    return f"{summary}\n"

# 데이터 전처리 함수
def preprocess_function(examples):
    # instruction과 completion 생성
    prompts = [create_prompt_template(doc) for doc in examples["document"]]
    completions = [create_completion_template(summary) for summary in examples["summary"]]

    # 전체 텍스트 생성 (instruction + completion)
    texts = [f"[INST]{prompt}[/INST]{completion}" for prompt, completion in zip(prompts, completions)]

    # 토큰화
    tokenized = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors=None
    )

    # labels 설정 (input_ids와 동일)
    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized

# 데이터셋 전처리 적용
encoded_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)

In [10]:
# 학습/검증 데이터 분할
train_val = encoded_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_val["train"]
eval_dataset = train_val["test"]

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(eval_dataset)}")

Train dataset size: 3200
Validation dataset size: 800


In [ ]:
# wandb 설정
wandb.login(key='****')
run = wandb.init(project='Korean News Summarization', job_type='training', anonymous='allow')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dgriii0606 (dg-test) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [12]:
import bitsandbytes
# 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# LoRA 설정
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"],
    inference_mode=False,
    init_lora_weights="gaussian"
)

# 모델 설정
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"": 0},
    trust_remote_code=True,
    torch_dtype=torch.float16  # float16으로 설정하여 메모리 효율성 향상
)

# 모델 준비
model.config.use_cache = False  # gradient checkpointing을 위해 cache 비활성화
model.enable_input_require_grads()  # 입력에 대한 그래디언트 활성화

# 모델의 pad_token_id도 동일하게 설정
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id

# LoRA 적용
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# 학습 가능한 파라미터 확인
print("Trainable parameters:", {
    name: param.shape for name, param in model.named_parameters() if param.requires_grad
})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Trainable parameters: {'base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight': torch.Size([8, 4096]), 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight': torch.Size([4096, 8]), 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight': torch.Size([8, 4096]), 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight': torch.Size([1024, 8]), 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight': torch.Size([8, 4096]), 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight': torch.Size([4096, 8]), 'base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight': torch.Size([8, 4096]), 'base_model.model.model.layers.1.self_attn.v_proj.lora_B.default.weight': torch.Size([1024, 8]), 'base_model.model.model.layers.2.self_attn.q_proj.lora_A.default.weight': torch.Size([8, 4096]), 'base_model.model.model.layers.2.self_attn.q_proj.lora_B.default.weight': torch.Size([4096, 8]), 'base_m

In [13]:
# 필요한 라이브러리 import
from transformers import (
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback
)

In [14]:
# 트레이너 설정
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,

    # 배치 크기 및 그래디언트 누적
    per_device_train_batch_size=2,  # 메모리 문제 방지를 위해 감소
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,  # 증가된 효과적 배치 크기

    # 학습률 및 스케줄링
    learning_rate=2e-4,            # 학습률 약간 증가
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,

    # 옵티마이저 설정
    optim="adamw_torch",          # paged_adamw_32bit 대신 기본 adamw 사용
    weight_decay=0.01,

    # 평가 및 저장 전략
    eval_strategy="steps",
    eval_steps=50,                # 더 자주 평가
    save_strategy="steps",
    save_steps=50,
    save_total_limit=3,

    # 학습 최적화
    fp16=True,
    gradient_checkpointing=True,

    # 로깅 설정
    logging_dir='./logs',
    logging_steps=10,
    report_to=["wandb"],

    # 기타 최적화
    dataloader_num_workers=2,     # 워커 수 감소
    group_by_length=True,
    remove_unused_columns=True,
    load_best_model_at_end=True,

    # 추가 설정
    ddp_find_unused_parameters=False,  # DDP 최적화
    torch_compile=False,              # 컴파일 비활성화로 안정성 향상
)

In [15]:
# 트레이너 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False  # Causal LM을 위해 False로 설정
    ),
    callbacks=[
        EarlyStoppingCallback(   # 조기 종료 설정
            early_stopping_patience=3,
            early_stopping_threshold=0.01
        )
    ]
)

In [16]:
# 학습 실행
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
50,1.386000,1.396932
100,1.351800,1.358920
150,1.330800,1.329764
200,1.346600,1.309978
250,1.278900,1.294523
300,1.243600,1.272925
350,1.248000,1.255258
400,1.228500,1.238671
450,1.161100,1.234641
500,1.167600,1.227961


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=550, training_loss=1.2647898119146173, metrics={'train_runtime': 3651.5049, 'train_samples_per_second': 2.629, 'train_steps_per_second': 0.164, 'total_flos': 1.923188678197248e+17, 'train_loss': 1.2647898119146173, 'epoch': 2.75})

In [22]:
# 테스트 데이터셋으로 모델 평가
!pip install rouge_score -qqq
from rouge_score import rouge_scorer
import numpy as np
from tqdm.auto import tqdm
import torch

def evaluate_model_metrics(model, tokenizer, test_dataset, num_samples=None):
    """모델 성능 평가"""

    # ROUGE 스코어 초기화
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # 평가 결과 저장용 리스트
    rouge_scores = {
        'rouge1': [],
        'rouge2': [],
        'rougeL': []
    }

    # 평가할 샘플 수 설정
    if num_samples is None:
        num_samples = len(test_dataset)
    else:
        num_samples = min(num_samples, len(test_dataset))

    # 평가 진행
    model.eval()
    for i in tqdm(range(num_samples)):
        # 인코딩되었던 eval_dataset의 input_ids를 원본 텍스트로 디코딩
        document = tokenizer.decode(test_dataset[i]['input_ids'], skip_special_tokens=True)

        # 인코딩되었던 eval_dataset의 labels를 요약문으로 디코딩
        reference = tokenizer.decode(test_dataset[i]['labels'],  skip_special_tokens=True)

        # 모델 예측
        try:
            prompt = f"[INST]아래 뉴스 기사를 요약해주세요:\n기사 내용:\n{document}\n요약:[/INST]"
            inputs = tokenizer([prompt], return_tensors="pt").to("cuda:0")

            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=128,
                    do_sample=True,
                    temperature=0.7
                )

            predicted = tokenizer.decode(outputs[0], skip_special_tokens=True)
            predicted = predicted.split("[/INST]")[-1].strip()

            # ROUGE 점수 계산
            scores = scorer.score(reference, predicted)
            rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
            rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
            rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

        except Exception as e:
            print(f"Error processing sample {i}: {str(e)}")
            continue

    # 평균 점수 계산
    avg_scores = {
        'rouge1': np.mean(rouge_scores['rouge1']),
        'rouge2': np.mean(rouge_scores['rouge2']),
        'rougeL': np.mean(rouge_scores['rougeL'])
    }

    # 결과 출력
    print("\n=== 모델 평가 결과 ===")
    print(f"평가한 샘플 수: {len(rouge_scores['rouge1'])}")
    print(f"ROUGE-1: {avg_scores['rouge1']:.4f}")
    print(f"ROUGE-2: {avg_scores['rouge2']:.4f}")
    print(f"ROUGE-L: {avg_scores['rougeL']:.4f}")

    return avg_scores

# 사용 예시
scores = evaluate_model_metrics(model, tokenizer, eval_dataset, num_samples=100)

  0%|          | 0/100 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o


=== 모델 평가 결과 ===
평가한 샘플 수: 100
ROUGE-1: 0.3460
ROUGE-2: 0.2293
ROUGE-L: 0.3380


In [23]:
# 모델 저장
trainer.model.save_pretrained(new_model)

In [24]:
# 저장한 모델과 토크나이저 허깅페이스에 업로드
# LoRA 어댑터 결합 및 허깅페이스 허브 업로드
import torch
from peft import LoraConfig, PeftModel

model_name = base_model  # 위에서 지정한 기본 모델
device_map = {"": 0}  # 모델을 로드할 디바이스 설정  # 0: 모델의 모든 파라미터를 GPU 0번 디바이스에 로드

# LoRA와 기초 모델 파라미터 합치기
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,      # CPU 메모리 사용을 최소화하여 모델을 로드
    return_dict=True,            # 모델 출력을 dictionary 형식으로 반환
    torch_dtype=torch.float16,   # 16비트 부동소수점 형식으로 로드(메모리 사용량 감소)
    device_map=device_map,       # GPU 0번 디바이스에 모델을 로드
)
model = PeftModel.from_pretrained(base_model, new_model)  # PEFT를 통해 미세 조정된 LoRA 어댑터를 로드하여 기본 모델(base_model)에 결합
model = model.merge_and_unload()   # LoRA 어댑터 파라미터를 기본 모델에 병합, 병합 후 로라 어댑터는 메모리에서 언(un)로드

# 허깅페이스 허브에 모델 및 토크나이저 저장
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)  # 모델 및 토크나이저 허깅페이스 허브에 업로드  # 임시 디렉토리 지정안하고 현재 경로에서 업로드

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/edgeun/mistral-7b-konews-summarizer/commit/6980971e3e0b25f194a9c9ba6abde19464817ab3', commit_message='Upload tokenizer', commit_description='', oid='6980971e3e0b25f194a9c9ba6abde19464817ab3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/edgeun/mistral-7b-konews-summarizer', endpoint='https://huggingface.co', repo_type='model', repo_id='edgeun/mistral-7b-konews-summarizer'), pr_revision=None, pr_num=None)

In [3]:
# 업로드한 모델과 토크나이저 불러와서 추론하기
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
import torch

new_model_name = "edgeun/mistral-7b-konews-summarizer"

# 베이스 모델의 토크나이저와 파인튜닝된 모델 로드
tokenizer = AutoTokenizer.from_pretrained(new_model_name)
model = AutoModelForCausalLM.from_pretrained(new_model_name).to("cuda:0")

# 토크나이저 설정
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [11]:
# 추론 함수 생성
def stream_summary(article):
    prompt = f"[INST]아래 뉴스 기사를 요약해주세요:\n기사 내용:\n{article}\n요약:[/INST]"
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda:0")
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=256, do_sample=True, temperature=0.7)

In [12]:
# 기사 텍스트 입력 및 함수 사용
new_article = """

정월대보름(2월12일)을 앞두고 서민들의 시름이 깊어지고 있다.
고물가 장기화에 장바구니 가격 부담이 갈수록 커지는 가운데 정월대보름에 챙겨먹는 오곡밥과 부럼 등 재료 가격마저 크게 올랐기 때문이다. 대형마트는 국산 재료 값이 급등하자 일부 품목을 수입산으로 대체하고 있다.
10일 대형마트 업계에 따르면 오곡밥 주재료인 붉은팥, 찹쌀, 서리태, 수수, 차조 등 국산 잡곡 시세가 일제히 상승했다.
특히 잡곡밥에 들어가는 붉은팥 가격이 전년 대비 50%가량 뛰었고 찹쌀도 23% 이상 급등했다. 부럼 재료인 은행과 땅콩 가격 역시 17%가량 올랐다. 국산 건나물도 상황은 마찬가지다.
호박과 고구마순의 가격이 각각 20%, 10% 이상 뛰었고 기획상품으로 내놓는 건나물 4종 세트 역시 전년 대비 평균 5~10% 올랐다.
유통업계에서는 정월대보름 주요 품목 가격이 오른 이유로 재배 면적 축소에 따른 생산량 감소, 폭염 등 이상기후로 인한 작황 부진, 고물가 장기화에 집밥 수요 급증 등의 영향 등을 꼽고 있다.
이에 따라 대형마트들은 고객들의 장바구니 물가 부담을 덜어주기 위해 일부 품목을 수입산으로 대체하고 있다.
붉은팥과 호두, 땅콩 등이 대표적이다.
롯데마트는 오는 12일까지 캐나다·페루산 붉은팥을 1㎏·1.2㎏들이 1봉당 각각 7990원에 선보인다.
또 중국산 볶음 피땅콩(450g)과 미국산 피호두(미국산·300g)를 2개 이상 구매하면 개당 2000원 할인한 각각 5990원에 판다.
홈플러스는 같은 기간 캐나다산 붉은팥을 1봉(600ｇ)당 9990원에 팔고 1봉을 사면 1봉을 덤으로 주는 ‘1+1’ 행사를 연다.
또 미국산 호두(300g)와 중국산 볶음피땅콩(500g)을 4990원에 내놓고, 미국산 피스타치오(100g)를 포함, 대보름 부럼세트(총 280ｇ)를 6990원에 판매한다.
이마트 역시 미국산 피호두(300ｇ)와 중국산 볶음 피땅콩(480ｇ)을 25% 할인한 5235원에 내놓는다.
한편 한국물가정보가 발표한 정월 대보름 주요 10개 품목 가격을 보면 전통시장의 합산 가격은 지난해 대비 6.2% 오른 13만9700원, 대형마트는 8.0% 오른 18만5220원으로 나타났다.
가격 상승 폭이 가장 큰 품목은 오곡밥 재료 중 붉은팥으로 1되(800g)가 전통시장에서는 전년 대비 45.5% 오른 1만6000원, 대형마트에서는 45.0% 오른 2만1920원이었다.
찹쌀은 1되(800g) 가격이 전통시장 기준 3200원으로 지난해보다 23.1% 올랐고, 대형마트에서는 5040원으로 28.6% 뛰었다.
검정콩 1되(720g)는 지난해보다 전통시장·대형마트 판매 가격이 각각 7.1%, 5.2% 올랐다.
부럼 재료 중에서는 은행과 땅콩 가격이 크게 올랐다. 은행 1되(600g) 가격은 전통시장 7000원, 대형마트 9840원으로 지난해보다 각각 16.7%, 15.2% 뛰었다.
땅콩 1되(400g)는 전통시장 가격이 1만원으로 지난해보다 11.1% 올랐고, 대형마트는 1만3560원으로 13.4% 인상됐다.

"""
stream_summary(new_article)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


10일 대형마트 업계에 따르면 국산 재료 가격이 급등하자 일부 품목을 수입산으로 대체하고 있다. 농축물가 상승 폭이 가장 큰 품목은 오곡밥 재료 중 붉은팥으로 1되(800g)가 전년 대비 45.5% 오른 1만6000원, 찹쌀은 1되(800g)가 23.1% 올랐다. 찹쌀은 1되(800g)는 지난해보다 전통시장 가격이 3200원으로 23.1% 올랐고, 대형마트에서는 5040원으로 28.6% 뛰었다. 부럼 
